# Example 3: Visualize all illustrations for a given publisher

In this case, the firm Munroe & Francis, active in Boston from c. 1800 to 1860.

## Step 1: Search Hathifile for publisher

Hathifiles can be very big, so we iteratively search them for field (column) values matching a query. This can take some finesse, since publisher names are often very similar and the name of a firm can be written in slightly different ways (e.g. '&' vs. 'and').

In [1]:
import pandas as pd
import numpy as np
import os, random, re, sys
from glob import glob
from annoy import AnnoyIndex

In [2]:
# the volumes used in the ACS project
HATHIFILE = "google_ids_1800-1850.txt.gz"
HATHICOLS = "hathifiles/hathi_field_list.txt"

In [3]:
def search_hathifile(ht_file, col_file, search_col, search_expr):
    """
    Given a hathifile and field names, return dataframe of rows
    where search_col contains search_expr (a regex)
    """
    # Use iterative method to scale to full hathifiles
    with open(col_file, "r") as fp:
        col_names = fp.readline().strip('\n').split('\t')
        num_cols = len(col_names)

    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
    iter_csv = pd.read_csv(
        ht_file, 
        sep='\t', 
        header=None,
        names=col_names,
        engine='c',
        # quicker if we can assert some types for the fields
        dtype={
            'htid': 'str',
            'rights_date_used': 'object', # values NOT guaranteed to be numeric
            'pub_place': 'str', # sadly, this is just the partner lib
            'imprint': 'str'
        },
        iterator=True,
        chunksize=5000,
        error_bad_lines=False)

    df = pd.DataFrame()
    for i, chunk in enumerate(iter_csv):
        condition = (chunk[search_col].str.contains(search_expr, na=False, flags=re.IGNORECASE))
        
        # hathifile idx has no relation to Neighbor tree: ignore
        df = pd.concat([df, chunk[condition]], ignore_index=True)
        
    return df

In [4]:
# find publishers "Munroe, Francis", "Munroe and Francis", "Munroe & Francis" (with matching group)
search_col = 'imprint'
search_expr = r"\bMunroe(?:,| and| &) Francis\b"

# a label for the results of this experiment (in case you want to compare later)
search_label = "munroe-francis"

df = search_hathifile(HATHIFILE, HATHICOLS, search_col, search_expr)
df.shape

(360, 26)

In [5]:
df.columns # use title, rights_date_used, imprint

Index(['htid', 'access', 'rights', 'ht_bib_key', 'description', 'source',
       'source_bib_num', 'oclc_num', 'isbn', 'issn', 'lccn', 'title',
       'imprint', 'rights_reason_code', 'rights_timestamp', 'us_gov_doc_flag',
       'rights_date_used', 'pub_place', 'lang', 'bib_fmt', 'collection_code',
       'content_provider_code', 'responsible_entity_code',
       'digitization_agent_code', 'access_profile_code', 'author'],
      dtype='object')

In [6]:
df['title'].head(5)

0    The Monthly anthology, and Boston review.
1    The Monthly anthology, and Boston review.
2    The Monthly anthology, and Boston review.
3    The Monthly anthology, and Boston review.
4    The Monthly anthology, and Boston review.
Name: title, dtype: object

In [7]:
# convert date objects to ints
df['rights_date_used'] = pd.to_numeric(df['rights_date_used']).astype(int)

In [8]:
# show a few results -- just the search field and the date published
df[[search_col, 'rights_date_used']].head(10)

,imprint,rights_date_used
0,Munroe and Francis [etc.],1804
1,Munroe and Francis [etc.],1807
2,Munroe and Francis [etc.],1808
3,Munroe and Francis [etc.],1809
4,Munroe and Francis [etc.],1809
5,Munroe and Francis [etc.],1805
6,Munroe and Francis [etc.],1806
7,Munroe and Francis [etc.],1810
8,Munroe and Francis [etc.],1810
9,Munroe and Francis [etc.],1811


## Step 2: Find search result matches in illustration metadata

We have a bunch of `htid`s from the Hathifile, but many of them will not contain any illustrations. To narrow down our set of results, we need to look up the `htid`s in our illustration metadata. This can be done with the main CSV file or with the vectors.tar file. Either way, the goal is to get a list of all image or vector files corresponding to specific regions of interest (illustrations) for the volumes returned in our search.

If you want to work with the vectors in `vectors.tar`, you will want to convert to HTRCs stubbytree format.

In [9]:
# Utility functions from Hathi's feature datasets
# https://github.com/htrc/htrc-feature-reader/blob/39010fd41c049f4f86b9c8ff4a44e000217093c2/htrc_features/utils.py
def _id_encode(id):
    '''
    :param id: A Pairtree ID. If it's a Hathitrust ID, this is the part after the library
        code; e.g. the part after the first period for vol.123/456.
    :return: A sanitized id. e.g., 123/456 will return as 123=456 to avoid filesystem issues.
    '''
    return id.replace(":", "+").replace("/", "=").replace(".", ",")

def _id_decode(id):
    '''
    :param id: A sanitized Pairtree ID.
    :return: An original Pairtree ID.
    '''
    return id.replace("+", ":").replace("=", "/").replace(",", ".")

def clean_htid(htid):
    '''
    :param htid: A HathiTrust ID of form lib.vol; e.g. mdp.1234
    :return: A sanitized version of the HathiTrust ID, appropriate for filename use.
    '''
    libid, volid = htid.split('.', 1)
    volid_clean = _id_encode(volid)
    return '.'.join([libid, volid_clean])

def id_to_stubbytree(htid, format = None, suffix = None, compression = None):
    '''
    Take an HTRC id and convert it to a 'stubbytree' location.
    '''
    libid, volid = htid.split('.', 1)
    volid_clean = _id_encode(volid)

    suffixes = [s for s in [format, compression] if s is not None]
    filename = ".".join([clean_htid(htid), *suffixes])
    path = os.path.join(libid, volid_clean[::3], filename)
    return path

In [10]:
# keep a mapping from unencoded htids from the hathifile...
stubby_dict = {id_to_stubbytree(htid): htid for htid in df.htid.values}

In [11]:
stubby_dict

{'mdp\\31331\\mdp.39015038731918': 'mdp.39015038731918',
 'mdp\\31197\\mdp.39015010791476': 'mdp.39015010791476',
 'mdp\\31198\\mdp.39015010791484': 'mdp.39015010791484',
 'mdp\\31199\\mdp.39015010791492': 'mdp.39015010791492',
 'mdp\\31190\\mdp.39015010791500': 'mdp.39015010791500',
 'mdp\\31678\\mdp.39015065471685': 'mdp.39015065471685',
 'mdp\\31677\\mdp.39015065471677': 'mdp.39015065471677',
 'mdp\\31676\\mdp.39015065471669': 'mdp.39015065471669',
 'mdp\\31675\\mdp.39015065471651': 'mdp.39015065471651',
 'mdp\\31674\\mdp.39015065471644': 'mdp.39015065471644',
 'uc1\\a035\\uc1.aa0001388586': 'uc1.aa0001388586',
 'mdp\\31168\\mdp.39015012365683': 'mdp.39015012365683',
 'hvd\\hd\\hvd.hn5drs': 'hvd.hn5drs',
 'mdp\\31683\\mdp.39015069385030': 'mdp.39015069385030',
 'uc1\\$01\\uc1.$b309318': 'uc1.$b309318',
 'hvd\\34924\\hvd.32044092528744': 'hvd.32044092528744',
 'hvd\\34922\\hvd.32044092528728': 'hvd.32044092528728',
 'hvd\\34921\\hvd.32044092528710': 'hvd.32044092528710',
 'hvd\\34923

In [12]:
# N.B. this assumes the vectors.tar file has been extracted to this directory
# You also need to be careful about filepath conventions; best to use os.path
VEC_DIR = os.path.abspath("../_app-files/roi-vectors/vectors")

In [13]:
# for each volume, find associated .npy vectors within stubbytree directory -- store in dictionary
munroe_francis = {}

for stubby_id in stubby_dict.keys():
    vol_path = os.path.join(VEC_DIR, stubby_id + "*.npy")
    vol_vectors = glob(vol_path)
    if len(vol_vectors) != 0:
        munroe_francis[stubby_id] = vol_vectors

In [16]:
# columns we want to keep from hathifile: these will map to 'Description' and 'Year' in PixPlot
col_map = {
    'rights_date_used': 'Year',
    'title': 'Description'
}

rows = []
for k, v in munroe_francis.items():
    
    # transform .npy file into jpeg, separate from rest of path
    for npy_file in v:
        
        vec_base = os.path.basename(npy_file)
        img_base = os.path.splitext(vec_base)[0] + '.jpg'
        
        # remember the unencoded htid
        htid = stubby_dict[k]
        
        # row to be added to df_pixplot
        row = {}
        
        # get metadata for this volume
        metadata = df[df['htid'] == htid][col_map.keys()]
        
        # tricky, since values could be a list or object
        for col in metadata.columns:
            row[col_map[col]] = metadata[col].values[0]

        # add img_base path and label
        row['Filename'] = img_base
        row['Label'] = search_label
        
        rows.append(row)

In [17]:
# turn dict rows into dataframe and write out to CSV
df_pixplot = pd.DataFrame.from_dict(rows)
df_pixplot.tail(5)

,Year,Description,Filename,Label
1472,1846,Peter Parley's book of Bible stories for child...,hvd.hwrcv7_00000255_00.jpg,munroe-francis
1473,1836,The year book : an astronomical and philosophi...,uc1.b3082741_00000006_00.jpg,munroe-francis
1474,1836,The year book : an astronomical and philosophi...,uc1.b3082741_00000254_00.jpg,munroe-francis
1475,1836,The year book : an astronomical and philosophi...,nyp.33433112037308_00000012_00.jpg,munroe-francis
1476,1836,The year book : an astronomical and philosophi...,nyp.33433112037308_00000260_00.jpg,munroe-francis


In [18]:
# use the search label to make a metadata path
metadata_csv = "{}_metadata.csv".format(search_label)

# PixPlot reqs: some of these are defaults, but just to be clear
df_pixplot.to_csv(metadata_csv, sep=',', header=True, index=False)

### Step 2a (optional): Create Annoy index using project vectors

You can experiment with building a smaller Annoy index with just these results.

In [ ]:
# Modified from: https://github.com/spotify/annoy
f = 1000
t = AnnoyIndex(f, 'angular')
i = 0

# Find all vectors per volume and index them from 0
for k,v in munroe_francis.items():
    for vec in v:
        item = np.load(vec)
        # transpose vector since it needs to be (1000,1) not (1,1000)
        t.add_item(i, item.T)
        i += 1

# Try with 1000 trees
t.build(1000)
t.save('munroe-francis.ann')

In [ ]:
u = AnnoyIndex(f, 'angular')
u.load('munroe-francis.ann')
print(u.get_nns_by_item(0, 10))

## Step 3: Visualize images from search results

Now that we have a reasonably sized subset matching our search query, we can visualize the ROIs belonging to those volumes.

WARNING: we are still figuring out how to access the raw image data. I plan to write a fallback method that uses the Hathi Data API.

In [ ]:
ROI_DIR = "..\_app-files\munroe-francis\samplecrops"

In [ ]:
# we know that each subdirectory is an htid, with JPEG crops inside it
roi_jpegs = glob(ROI_DIR + '/*/*.jpg')
len(roi_jpegs)

In [ ]:
# just use pixplot: https://github.com/YaleDHLab/pix-plot
# do a quick pip freeze of environment htrc (conda list as well)
# remember to include metadata!